In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.client.syncing import compare_states
from syft.client.syncing import resolve

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/eelco/dev/PySyft


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=a977febb3ad745cbba47b4174b2390a6 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=7fb1a7f3d6914318a4bb3a7c6ce79465 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.64it/s]

Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 37.85it/s]

Uploading: numpy-data


# Make Requests

In [8]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    print("Computing mean...")
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/5e82e3049587441e9f9f28cc17b999aa.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=2 image_uid=eb1a352b7fb84dd9891e4ec66092c6ed in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_49: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_49
override True
Computing mean...
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [9]:
client_low_ds.code.request_code_execution(compute_mean)

syft.service.request.request.Request

## Sync to high side

In [10]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

SyftInfo: Node Landed!


In [11]:
low_state

syft.service.sync.sync_state.SyncState

In [12]:
diff_state = compare_states(low_state, high_state)

In [13]:
diff_state

syft.service.sync.diff_state.NodeDiff

In [14]:
resolved_state_low, resolved_state_high = resolve(diff_state, decision="low")

LOW SIDE STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #a058d733a0dc447cb85984133cc3a0e8:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]
worker_pool_name: default-pool

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #a600fa22bd134f969933660af62385be:
    approved: False
    status_dict: {
    ‎ ‎ ‎ ‎ node_id=<UID: 8a1c04544655402190588aec30079bc3>
        verify_key=1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367
        node_name='test_l': (<UserCodeStatus.PENDING: 'pending'>, '')
    }



HIGH SIDE STATE:

No high side changes.

Decision: Syncing 2 objects from low side


LOW SIDE STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

REQUEST #829e23e5e0044312ac8a3992e00b9136:
request_ti

In [15]:
print("Resolved state low side")
print(resolved_state_low)
print()
print("Resolved state high side")
print(resolved_state_high)

Resolved state low side
ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)

Resolved state high side
ResolvedSyncState(
  create_objs=[{NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}, syft.service.code.user_code.UserCode, syft.service.request.request.Request],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)


In [16]:
client_low.apply_state(resolved_state_low)

SyftSuccess: Synced 0 items

In [17]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 3 items

# Run code high and sync back result

In [18]:
data_high = client_high.datasets[0].assets[0]

In [19]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = 4b93928e22894c9cbe4748608d4e41df
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [20]:
# wait for the result
job_high.wait().get()

override True


04/03/24 10:08:57 FUNCTION LOG (4b93928e22894c9cbe4748608d4e41df): Computing mean...


17.0

In [21]:
job_info = job_high.info(public_metadata=True, result=True)

request = client_high.requests[0]
result_obj = job_high.result

In [22]:
# syft absolute
from syft import SyftError
from syft import SyftSuccess

# Accepting the result directly gives an error
accept_res = request.accept_by_depositing_result(result_obj)
assert isinstance(accept_res, SyftError)

In [23]:
accept_res = request.accept_by_depositing_result(job_info)

assert isinstance(accept_res, SyftSuccess)
accept_res

Approving request for domain test_h
returning existing job
setting permission
None
None
Approving request for domain test_h
ADDING PERMISSION [READ: 14bc84f5915d475e8e1113f52d5607aa as b54e2bd21b45ccf1c1ee02a7b37306bf13753d8043d56af2a5c92e40c54df856] 14bc84f5915d475e8e1113f52d5607aa
New result 17.0
Job(4b93928e22894c9cbe4748608d4e41df) Setting new result 14bc84f5915d475e8e1113f52d5607aa -> 14bc84f5915d475e8e1113f52d5607aa


SyftSuccess: Request submitted for updating result.

In [24]:
request.code.output_history[0]

```python
class ExecutionOutput:
  id: str = 78422d78c11f452c9dcc5b7e433c87f9
  created_at: str = 2024-03-04 08:08:45
  user_code_id: str = a058d733a0dc447cb85984133cc3a0e8
  job_id: str = 4b93928e22894c9cbe4748608d4e41df
  output_ids: str = [<LineageID: 14bc84f5915d475e8e1113f52d5607aa - 1438121221781734051>]

```

In [25]:
# accept_res = request.accept_by_depositing_result(job_info.result.get())

# assert isinstance(accept_res, SyftSuccess)
# accept_res

In [26]:
# Need to refresh Job because we overwrite result
job_high = client_high.jobs[0]

action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service(
    "blobstorageservice"
).stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_high.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
)

## Sync back to low

In [27]:
client_high.jobs[0]

```python
class Job:
    id: UID = 4b93928e22894c9cbe4748608d4e41df
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 Computing mean...
JOB COMPLETED
    
```

In [32]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

high_state

syft.service.sync.sync_state.SyncState

In [29]:
diff_state_2 = compare_states(low_state, high_state)

diff_state_2

syft.service.sync.diff_state.NodeDiff

In [33]:
# syft absolute
from syft import ActionObject

In [34]:
resolved_state_low, resolved_state_high = resolve(
    diff_state_2, decision="high", share_private_objects=True
)

LOW SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #a058d733a0dc447cb85984133cc3a0e8:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]

    ―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #a600fa22bd134f969933660af62385be:
    status_dict: (<UserCodeStatus.PENDING: 'pending'>, '')



HIGH SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #a058d733a0dc447cb85984133cc3a0e8:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_h, Status: approved
]

    ―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #a600fa22bd134f969933660af62385be:
    status_dict: (<UserCodeStatus.AP

In [35]:
print(resolved_state_low)
print()
print(resolved_state_high)

ResolvedSyncState(
  create_objs=[syft.service.output.output_service.ExecutionOutput, Pointer:
17.0, syft.service.job.job_stash.Job, syft.service.log.log.SyftLog],
  update_objs=[{NodeIdentity <name=test_h, id=8212e679, 🔑=d311a667>: (<UserCodeStatus.APPROVED: 'approved'>, '')}, syft.service.request.request.Request],
  delete_objs=[]
  new_permissions=[[READ: 14bc84f5915d475e8e1113f52d5607aa as b54e2bd21b45ccf1c1ee02a7b37306bf13753d8043d56af2a5c92e40c54df856], [READ: 14bc84f5915d475e8e1113f52d5607aa as b54e2bd21b45ccf1c1ee02a7b37306bf13753d8043d56af2a5c92e40c54df856], [READ: 4b93928e22894c9cbe4748608d4e41df as b54e2bd21b45ccf1c1ee02a7b37306bf13753d8043d56af2a5c92e40c54df856], [READ: 4b93928e22894c9cbe4748608d4e41df as b54e2bd21b45ccf1c1ee02a7b37306bf13753d8043d56af2a5c92e40c54df856], [READ: 4505fb7d99f8449b8dd25663ac43b714 as b54e2bd21b45ccf1c1ee02a7b37306bf13753d8043d56af2a5c92e40c54df856], [READ: 4505fb7d99f8449b8dd25663ac43b714 as b54e2bd21b45ccf1c1ee02a7b37306bf13753d8043d56af2a5c92

In [36]:
client_low.apply_state(resolved_state_low)

SyftSuccess: Synced 6 items

In [37]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 0 items

In [38]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

# Run code low

## Run

In [39]:
client_low_ds._fetch_api(client_low.credentials)

In [40]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low.get()

override False


17.0

In [41]:
res_low.get()

17.0

In [42]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id == job_high.result.id.id == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [43]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low.get()

override False


17.0

In [44]:
res_low.get()

17.0

In [45]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id == job_high.result.id.id == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [46]:
private_high.mean()

17.0

In [47]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)
job_low

SyftWarning: There are existing jobs for this user code, returning the latest one

```python
class Job:
    id: UID = 4b93928e22894c9cbe4748608d4e41df
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 Log 4505fb7d99f8449b8dd25663ac43b714 not available
JOB COMPLETED
    
```

In [48]:
job_low.wait().get()

17.0

In [49]:
job_low.logs()

Log 4505fb7d99f8449b8dd25663ac43b714 not available


In [50]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert (
    job_low.result.syft_blob_storage_entry_id
    == job_high.result.syft_blob_storage_entry_id
)